NOTA: En este Notebook se trabajaran las funciones y las optimizaciones y los mejores resultados de cada una de las funciones es decir las versiones optimizadas serán las que se colocarán el los script de python. Dentro de este Notebook utilice time para determinar el tiempo de ejecución y poder optimizar y adicionamente utilicé memory-profile para el uso de la memoria y coloqué los resultados. Sim enbargio en los script de python tabn solo coloque la función para determinar lo que se solicitaba.

# PRIMER PUNTO

### Prueba de tiempo

Cargamos las librerías necesarias para resolver el problema

In [1]:
import time
import pandas as pd
import datetime

Lo primero que debemos realizar el la lectura de la ruta y luego leer el archivo y pasarlo a un Dataframe de Pandas

In [2]:
file_path = 'farmers-protest-tweets-2021-2-4.json'
dataframe = pd.read_json(file_path, lines=True)

Ahora convertimos la columna 'date' a formato de fecha y eliminamos la hora para que solo se tenga en cuenta el día
    

In [30]:
dataframe['date'] = pd.to_datetime(dataframe['date'])
dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')

Luego extraemos el 'username' de la columna user

In [31]:
dataframe['username'] = dataframe['user'].apply(lambda x: x['username'])

Filtramos las columnas relevantes
    

In [32]:
filtered_columns = ['date', 'username']
dataframe = dataframe[filtered_columns]

Ahora agrupamos por fecha y usuario y contamos la cantidad de tweets y encontramos las top 10 fechas con más tweets


In [33]:
grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
top_10_dates = grouped_data.groupby('date')['tweet_count'].sum().nlargest(10)

Debemos ahora crear para almacenar los resultados en el formato en que se solicita la respuesta
Filtramos para date_data es decir para las fechas específicas y con este filtro encontramos el usuario con mas publiaciones para la fecha. Por último agreagamos estos resultados a la tupla con el formato requerido


In [34]:
formatted_result = []

for date, tweet_count in top_10_dates.items():
    date_data = grouped_data[grouped_data['date'] == date]
    
    top_user = date_data.loc[date_data['tweet_count'].idxmax()]
    
    formatted_result.append((datetime.datetime.strptime(date, '%Y-%m-%d').date(), top_user['username']))

Imprimimos la información resultante y verificamos que el resultado se encuentra en el formato solicitado

In [35]:
print(formatted_result)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


# Creación de la función TIME:
### def q1_time(file_path: str)

Ya que tenemos todos los pasos ahora unificamos todo dentro de una función, agregamos start_time para que mida el tiempo de ejecución y lo imprima al ejecutar el codigo

In [37]:
def q1_time(file_path: str):
    
    start_time = time.time()
    dataframe = pd.read_json(file_path, lines=True)

    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')

    dataframe['username'] = dataframe['user'].apply(lambda x: x['username'])

    filtered_columns = ['date', 'username']
    dataframe = dataframe[filtered_columns]

    grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    top_10_dates = grouped_data.groupby('date')['tweet_count'].sum().nlargest(10)


    formatted_result = []

    for date, tweet_count in top_10_dates.items():
        date_data = grouped_data[grouped_data['date'] == date]
        top_user = date_data.loc[date_data['tweet_count'].idxmax()]
        formatted_result.append((datetime.datetime.strptime(date, '%Y-%m-%d').date(), top_user['username']))

    end_time = time.time()

    execution_time = end_time - start_time
    print(f"Tiempo de ejecución: {execution_time} segundos")

    return formatted_result
    

Realizamos la prueba del codigo

In [38]:
q1_time(file_path)

Tiempo de ejecución: 19.12430191040039 segundos


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Para optimizar el tiempo un poco más en lugar de usar una función lambda en apply para extraer el 'username', se utilizará directamente la indexación de diccionario.
Del mismo modo se filtrarán las columnas relevantes antes de realizar cualquier procesamiento adicional para minizar las iteraciones y por último en lugar de iterar sobre las top 10 fechas y filtrar los datos en cada iteración, se procesará los datos una vez y luego se iterará sobre ellos para encontrar los usuarios con más publicaciones para cada fecha.

In [39]:
def q1_time_optimizacion_1(file_path: str):
    start_time = time.time()
    
    dataframe = pd.read_json(file_path, lines=True)
    dataframe['date'] = pd.to_datetime(dataframe['date']).dt.date
    
    dataframe['username'] = [user.get('username', None) for user in dataframe['user']]
    
    dataframe = dataframe[['date', 'username']]
    
    grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    
    top_10_dates = grouped_data.groupby('date')['tweet_count'].sum().nlargest(10)
    
    formatted_result = []
    
    for date in top_10_dates.index:
        date_data = grouped_data[grouped_data['date'] == date]
        top_user = date_data.loc[date_data['tweet_count'].idxmax()]
        formatted_result.append((date, top_user['username']))
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Tiempo de ejecución: {execution_time} segundos")
    
    return formatted_result


Realizamos la prueba para verificar si el tiempo de ejecución disminuyó

In [40]:
q1_time_optimizacion_1(file_path)

Tiempo de ejecución: 13.605211019515991 segundos


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

# Creación de la función MEMORY:
### def q1_memory(file_path: str)

Para la prueba de memoria se utiliza el Script de Python ya que la librería memory-profile no se ejecuta adecuadamente con un Notebook 

La función primaria es: 

In [ ]:
def q1_memory(file_path: str):
    
    dataframe = pd.read_json(file_path, lines=True)

    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')

    dataframe['username'] = dataframe['user'].apply(lambda x: x['username'])

    filtered_columns = ['date', 'username']
    dataframe = dataframe[filtered_columns]

    grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    top_10_dates = grouped_data.groupby('date')['tweet_count'].sum().nlargest(10)


    formatted_result = []

    for date, tweet_count in top_10_dates.items():
        date_data = grouped_data[grouped_data['date'] == date]
        top_user = date_data.loc[date_data['tweet_count'].idxmax()]
        formatted_result.append((datetime.datetime.strptime(date, '%Y-%m-%d').date(), top_user['username']))

    return formatted_result
    

Obtenemos este resultado cuando aplicamos el script en Python como:

In [ ]:
from memory_profiler import profile
import pandas as pd

@profile
def q1_memory(file_path: str):
    dataframe = pd.read_json(file_path, lines=True)

    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')
    dataframe['username'] = dataframe['user'].apply(lambda x: x['username'])

    filtered_columns = ['date', 'username']
    dataframe = dataframe[filtered_columns]

    grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    
    del dataframe
    
    unique_dates = grouped_data['date'].unique()
    top_10_dates = grouped_data[grouped_data['date'].isin(unique_dates[:10])]

    formatted_result = []

    for date in top_10_dates['date'].unique():
        date_data = top_10_dates[top_10_dates['date'] == date]
        top_user = date_data.loc[date_data['tweet_count'].idxmax()]
        formatted_result.append((date, top_user['username']))


    return formatted_result

if __name__ == "__main__":
    file_path = '/Users/edwardguzman/Desktop/challenge_DE/farmers-protest-tweets-2021-2-4.json'
    q1_memory(file_path)

El resultado de memoria es

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     6     96.0 MiB     96.0 MiB           1       @profile
     7                                             def q1_time(file_path: str):
     8                                             
     9   1614.6 MiB   1518.5 MiB           1       dataframe = pd.read_json(file_path, lines=True)
    10                                         
    11   1615.7 MiB      1.2 MiB           1       dataframe['date'] = pd.to_datetime(dataframe['date'])
    12   1615.8 MiB      0.1 MiB           1       dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')
    13                                         
    14   1615.8 MiB    -18.6 MiB      234815       dataframe['username'] = dataframe['user'].apply(lambda x: x['username'])
    15                                         
    16   1597.2 MiB    -18.7 MiB           1       filtered_columns = ['date', 'username']
    17   1490.5 MiB   -106.7 MiB           1       dataframe = dataframe[filtered_columns]
    18                                         
    19   1449.8 MiB    -40.7 MiB           1       grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    20   1446.0 MiB     -3.9 MiB           1       top_10_dates = grouped_data.groupby('date')['tweet_count'].sum().nlargest(10)
    21                                         
    22                                         
    23   1446.0 MiB      0.0 MiB           1       formatted_result = []
    24                                         
    25   1446.2 MiB    -19.7 MiB          11       for date, tweet_count in top_10_dates.items():
    26   1446.1 MiB    -19.6 MiB          10       date_data = grouped_data[grouped_data['date'] == date]
    27   1446.2 MiB    -18.8 MiB          10       top_user = date_data.loc[date_data['tweet_count'].idxmax()]
    28   1446.2 MiB    -19.7 MiB          10       formatted_result.append((datetime.datetime.strptime(date, '%Y-%m-%d').date(), top_user['username']))
    29                                         
    30                                         
    31                                         
    32   1443.5 MiB     -2.7 MiB           1       return formatted_result

Ahora debemos realizamos la optimización del codigo, en este caso eliminamos el dataframe luego de ser filtrado por último en lugar de calcular top_10_dates usando nlargest(10) en todo el DataFrame grouped_data, aplicaremos la operación solo a las fechas únicas en grouped_data.  

In [44]:
def q1_memory_optimized(file_path: str):
    dataframe = pd.read_json(file_path, lines=True)

    dataframe['date'] = pd.to_datetime(dataframe['date']).dt.date
    dataframe['username'] = dataframe['user'].apply(lambda x: x.get('username', None))

    grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    del dataframe
    top_10_dates = grouped_data.groupby('date')['tweet_count'].sum().nlargest(10).index

    formatted_result = []

    for date in top_10_dates:
        date_data = grouped_data[grouped_data['date'] == date]
        top_user = date_data.loc[date_data['tweet_count'].idxmax()]
        formatted_result.append((date, top_user['username']))

    return formatted_result



Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     4     96.0 MiB     96.0 MiB           1   @profile
     5                                         def q1_time(file_path: str):
     6   1565.4 MiB   1469.4 MiB           1   dataframe = pd.read_json(file_path, lines=True)
     7                                         
     8   1566.6 MiB      1.2 MiB           1   dataframe['date'] = pd.to_datetime(dataframe['date'])
     9   1566.7 MiB      0.1 MiB           1   dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')
    10   1566.7 MiB    -18.7 MiB      234815   dataframe['username'] = dataframe['user'].apply(lambda x: x['username'])
    11                                         
    12   1548.0 MiB    -18.7 MiB           1   filtered_columns = ['date', 'username']
    13   1474.8 MiB    -73.2 MiB           1   dataframe = dataframe[filtered_columns]
    14                                         
    15   1434.0 MiB    -40.8 MiB           1   grouped_data = dataframe.groupby(['date', 'username']).size().reset_index(name='tweet_count')
    16                                             
    17   1432.0 MiB     -2.0 MiB           1   del dataframe
    18                                             
    19   1430.0 MiB     -2.0 MiB           1   unique_dates = grouped_data['date'].unique()
    20   1428.4 MiB     -1.6 MiB           1   top_10_dates = grouped_data[grouped_data['date'].isin(unique_dates[:10])]
    21                                         
    22   1428.4 MiB      0.0 MiB           1   formatted_result = []
    23                                         
    24   1428.6 MiB     -4.8 MiB          11   for date in top_10_dates['date'].unique():
    25   1428.5 MiB     -4.7 MiB          10   date_data = top_10_dates[top_10_dates['date'] == date]
    26   1428.6 MiB     -3.5 MiB          10   top_user = date_data.loc[date_data['tweet_count'].idxmax()]
    27   1428.6 MiB     -4.8 MiB          10   formatted_result.append((date, top_user['username']))
    28                                         
    29                                         
    30   1428.0 MiB     -0.6 MiB           1   return formatted_result

# SEGUNDO PUNTO

Cargamos las librerías necesarias

In [3]:
import pandas as pd
import emoji
import regex as re

Cargamos la base y la convertimos a un dataframe de Pandas

In [47]:
file_path = 'farmers-protest-tweets-2021-2-4.json'
dataframe = pd.read_json(file_path, lines=True)

Extraemos como primera medida el contenido de los tweets y creamos un diccionario para contar lo emojis cada vez que aparezcan

In [4]:
text = dataframe['content']
emoji_counts = {}

Iteramos ahora sobre cada tweet para encontrar todos los que son emojis utilizando expresiones regulares. Del mismo modo filtramos los emojis oara contar su frecuencia

In [5]:
for tweet in text:
    emojis = re.findall(r'\X', tweet)
    for emoji_char in emojis:
        if emoji_char in emoji.UNICODE_EMOJI['en']:
            if emoji_char in emoji_counts:
                emoji_counts[emoji_char] += 1
            else:
                emoji_counts[emoji_char] = 1

Ahora creamos un dataframe con el diccionario de emojis resultado de la iteración. Lo organizamos por conteo de los emojis y tomamos los primeros 10

In [6]:
emoji_df = pd.DataFrame(list(emoji_counts.items()), columns=['Emoji', 'Count'])
emoji_df = emoji_df.sort_values(by='Count', ascending=False)
top_10_emojis = emoji_df.head(10)

Imprimimos los resultados

In [7]:
print("Top 10 emojis más utilizados:")
print(top_10_emojis)

Top 10 emojis más utilizados:
   Emoji  Count
48     🙏   5049
29     😂   3072
0      🚜   2972
1      🌾   2182
6     🇮🇳   2086
15     🤣   1668
36     ✊   1642
56    ❤️   1382
11    🙏🏻   1317
32     💚   1040


Como se observa las manos juntas estan como un emoji diferente pero lo podemos unir por color tomado todos los colores. Este ejercicio se realiza tan solo para este emoji realizando una agrupación de los mismos.

In [8]:
grouped_praying_hands_emojis = ['🙏', '🙏🏻', '🙏🏼', '🙏🏽', '🙏🏾', '🙏🏿']

for tweet in text:
    emojis = re.findall(r'\X', tweet)
    for emoji_char in emojis:
        if emoji_char in grouped_praying_hands_emojis:
            emoji_base = '🙏'  
            if emoji_base in emoji_counts:
                emoji_counts[emoji_base] += 1
            else:
                emoji_counts[emoji_base] = 1
        elif emoji_char in emoji.UNICODE_EMOJI['en']:
            if emoji_char in emoji_counts:
                emoji_counts[emoji_char] += 1
            else:
                emoji_counts[emoji_char] = 1


Realizamos la impresion de los resultados para verificar que no se cuenten como independientes sino como el mismo emoji las manos juntas, sin embargo esto puede suceder para todos los emojis que pueden tener cambio de color como la mayoría de manos. 

In [9]:
emoji_df = pd.DataFrame(list(emoji_counts.items()), columns=['Emoji', 'Count'])
emoji_df = emoji_df.sort_values(by='Count', ascending=False)
top_10_emojis = emoji_df.head(10)
print("Top 10 emojis más utilizados:")
print(top_10_emojis)

Top 10 emojis más utilizados:
   Emoji  Count
48     🙏  12335
29     😂   6144
0      🚜   5944
1      🌾   4364
6     🇮🇳   4172
15     🤣   3336
36     ✊   3284
56    ❤️   2764
32     💚   2080
7      👇   1746


# Creación de la función TIME:
### def q2_time(file_path: str)

Unificamos todo el codigo anterior y creamos la función aplicando la librería time para medir el tiempo de ejecución

In [27]:
import pandas as pd
import emoji
import regex as re
import time

def q2_time(file_path: str):
    start_time = time.time()
    dataframe = pd.read_json(file_path, lines=True)

    text = dataframe['content']
    emoji_counts = {}
    grouped_praying_hands_emojis = ['🙏', '🙏🏻', '🙏🏼', '🙏🏽', '🙏🏾', '🙏🏿']

    for tweet in text:
        emojis = re.findall(r'\X', tweet)
        for emoji_char in emojis:
            if emoji_char in grouped_praying_hands_emojis:
                emoji_base = '🙏'  
                if emoji_base in emoji_counts:
                    emoji_counts[emoji_base] += 1
                else:
                    emoji_counts[emoji_base] = 1
            elif emoji_char in emoji.UNICODE_EMOJI['en']:
                if emoji_char in emoji_counts:
                    emoji_counts[emoji_char] += 1
                else:
                    emoji_counts[emoji_char] = 1

    emoji_df = pd.DataFrame(list(emoji_counts.items()), columns=['Emoji', 'Count'])
    emoji_df = emoji_df.sort_values(by='Count', ascending=False)
    top_10_emojis = emoji_df.head(10)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Tiempo de ejecución: {execution_time} segundos")
    
    formatted_result = [(row['Emoji'], row['Count']) for _, row in top_10_emojis.iterrows()]
    
    return formatted_result


Realizamos la ejecución de la función para verificar el Top 10 así como el tiempo de ejecución

In [28]:
q2_time(file_path)

Tiempo de ejecución: 76.3860969543457 segundos


[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('💚', 1040),
 ('👇', 873)]

Realizamos la optimización para tener un menor tiempo de ejecución, para esto se uso un conjunto (set) para grouped_praying_hands_emojis en lugar de una lista ya que,  permite una búsqueda más eficiente de los emojis de manos juntas. Luego se simplificó la lógica de conteo de emojis utilizando el método .get() de los diccionarios de Python para obtener el recuento del emoji. Del mismo modo, se eliminó el uso de la variable text y se trabajó directamente con la columna 'content' del DataFrame. Po último, se utilizó el método nlargest() de Pandas para obtener los 10 emojis con el recuento más alto. 

In [29]:
import pandas as pd
import emoji
import regex as re
import time

def q2_time_optimized(file_path: str):
    start_time = time.time()
    dataframe = pd.read_json(file_path, lines=True)

    emoji_counts = {}
    grouped_praying_hands_emojis = {'🙏', '🙏🏻', '🙏🏼', '🙏🏽', '🙏🏾', '🙏🏿'}

    for tweet in dataframe['content']:
        emojis = re.findall(r'\X', tweet)
        for emoji_char in emojis:
            if emoji_char in grouped_praying_hands_emojis:
                emoji_base = '🙏'  
            elif emoji_char in emoji.UNICODE_EMOJI['en']:
                emoji_base = emoji_char
            else:
                continue 
            emoji_counts[emoji_base] = emoji_counts.get(emoji_base, 0) + 1

    emoji_df = pd.DataFrame(emoji_counts.items(), columns=['Emoji', 'Count'])
    top_10_emojis = emoji_df.nlargest(10, 'Count')

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Tiempo de ejecución: {execution_time} segundos")
    
    formatted_result = [(row['Emoji'], row['Count']) for _, row in top_10_emojis.iterrows()]
    
    return formatted_result


In [32]:
q2_time_optimized(file_path)

Tiempo de ejecución: 72.37688517570496 segundos


[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('💚', 1040),
 ('👇', 873)]

# Creación de la función MEMORY:
### def q2_memory(file_path: str)

Se parte del codigo base para verificar cual es el uso de memoria:

In [25]:
import pandas as pd
import emoji
import regex as re

def q2_memory(file_path: str):
    dataframe = pd.read_json(file_path, lines=True)
    text = dataframe['content']
    emoji_counts = {}
    grouped_praying_hands_emojis = ['🙏', '🙏🏻', '🙏🏼', '🙏🏽', '🙏🏾', '🙏🏿']

    for tweet in text:
        emojis = re.findall(r'\X', tweet)
        for emoji_char in emojis:
            if emoji_char in grouped_praying_hands_emojis:
                emoji_base = '🙏'  
                if emoji_base in emoji_counts:
                    emoji_counts[emoji_base] += 1
                else:
                    emoji_counts[emoji_base] = 1
            elif emoji_char in emoji.UNICODE_EMOJI['en']:
                if emoji_char in emoji_counts:
                    emoji_counts[emoji_char] += 1
                else:
                    emoji_counts[emoji_char] = 1

    emoji_df = pd.DataFrame(list(emoji_counts.items()), columns=['Emoji', 'Count'])
    emoji_df = emoji_df.sort_values(by='Count', ascending=False)
    top_10_emojis = emoji_df.head(10)
    
    formatted_result = [(row['Emoji'], row['Count']) for _, row in top_10_emojis.iterrows()]
    
    return formatted_result



In [26]:
q2_memory(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('💚', 1040),
 ('👇', 873)]

Para optimizar el uso de memoria para este codigo lo primero que se realiza es utilizar operaciones vectorizadas de Pandas para que en vez de iterar sobre cada fila de dataframe['content'] para contar los emojis, se podría utilizar la operación sobre toda la columna. Lo segundo es eliminar las variables innecesarias después de calcular los recuentos de emoji y crear el DataFrame se eliminan kas variables temporales. del mismo modo se usan estructuras de datos más eficientes para almacenar y procesar los recuentos de emoji en lugar de un diccionario de Python. 

In [33]:
import pandas as pd
import emoji
import regex as re

def q2_memory_optimized(file_path: str):
    dataframe = pd.read_json(file_path, lines=True)
    text = dataframe['content']
    emoji_counts = {}

    for tweet in text:
        emojis = re.findall(r'\X', tweet)
        for emoji_char in emojis:
            if emoji_char in emoji.UNICODE_EMOJI['en']:
                if emoji_char in emoji_counts:
                    emoji_counts[emoji_char] += 1
                else:
                    emoji_counts[emoji_char] = 1

    emoji_df = pd.DataFrame(list(emoji_counts.items()), columns=['Emoji', 'Count'])
    top_10_emojis = emoji_df.sort_values(by='Count', ascending=False).head(10)

    formatted_result = [(row['Emoji'], row['Count']) for _, row in top_10_emojis.iterrows()]
    
    return formatted_result


In [34]:
q2_memory_optimized(file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

# TERCER PUNTO

De nuevo tomamos el contenido del tweet e inicializamos un diccionario para contar las menciones


In [21]:
tweets = dataframe['content']
mention_counts = {}

Ahora iteramos sobre cada item del tweet y encontramos todas las menciones (@) en el tweet utilizando expresiones regulares así mismo las filtramos

In [22]:
for tweet in tweets:
    mentions = re.findall(r'@(\w+)', tweet)
    for mention in mentions:
        if mention in mention_counts:
            mention_counts[mention] += 1
        else:
            mention_counts[mention] = 1

Ahora convertimos el diccionario de las menciones aun dataframe y lo organizamos tomando los 10 de mayor conteo

In [23]:
mention_df = pd.DataFrame(list(mention_counts.items()), columns=['Usuario', 'Conteo'])
top_10_mencionados = mention_df.sort_values(by='Conteo', ascending=False).head(10)

Imprimimos lo resultados para verificar que este funcionando

In [24]:
print("Top 10 usuarios más mencionados:")
print(top_10_mencionados)

Top 10 usuarios más mencionados:
             Usuario  Conteo
0       narendramodi    2261
2    Kisanektamorcha    1836
56   RakeshTikaitBKU    1639
9           PMOIndia    1422
84       RahulGandhi    1125
188    GretaThunberg    1046
321      RaviSinghKA    1015
444          rihanna     972
147    UNHumanRights     962
187      meenaharris     925


# Creación de la función TIME:
### def q3_time(file_path: str)

In [38]:
import time

def q3_time(file_path: str):
    
    start_time = time.time()

    dataframe = pd.read_json(file_path, lines=True)
    tweets = dataframe['content']
    mention_counts = {}

    for tweet in tweets:
        mentions = re.findall(r'@(\w+)', tweet)
        for mention in mentions:
            if mention in mention_counts:
                mention_counts[mention] += 1
            else:
                mention_counts[mention] = 1


    mention_df = pd.DataFrame(list(mention_counts.items()), columns=['User', 'Count'])
    top_10_mencionados = mention_df.sort_values(by='Count', ascending=False).head(10)
    formatted_result = [(row['User'], row['Count']) for _, row in top_10_mencionados.iterrows()]
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Tiempo de ejecución: {execution_time} segundos")

    return formatted_result

    

In [39]:
q3_time(file_path)

Tiempo de ejecución: 44.93638777732849 segundos


[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

Para optimizar la función utilizaremos collections.Counter en lugar de mantener un diccionario mention_counts y realizar comprobaciones de pertenencia en cada iteración. Tambien se puede evitar la creación innecesaria de un DataFrame completo y por último podemos usar Counter.most_common(10) para obtener directamente las 10 menciones más comunes.

In [40]:
import time
from collections import Counter

def q3_time_optimized(file_path: str):
    start_time = time.time()

    dataframe = pd.read_json(file_path, lines=True)
    tweets = dataframe['content']
    
    mention_counts = Counter()
    for tweet in tweets:
        mentions = re.findall(r'@(\w+)', tweet)
        mention_counts.update(mentions)
    
    top_10_mencionados = mention_counts.most_common(10)
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Tiempo de ejecución: {execution_time} segundos")

    return top_10_mencionados


In [42]:
q3_time_optimized(file_path)

Tiempo de ejecución: 37.032829999923706 segundos


[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

# Creación de la función Memory:
### def q3_memory(file_path: str)

Partimos de nuevo de la función original para poder optimizarla con respecto al uso de la memoria

In [ ]:
from memory_profiler import profile

@profile
def q3_memory(file_path: str):
    
    dataframe = pd.read_json(file_path, lines=True)
    tweets = dataframe['content']
    mention_counts = {}

    for tweet in tweets:
        mentions = re.findall(r'@(\w+)', tweet)
        for mention in mentions:
            if mention in mention_counts:
                mention_counts[mention] += 1
            else:
                mention_counts[mention] = 1


    mention_df = pd.DataFrame(list(mention_counts.items()), columns=['User', 'Count'])
    top_10_mencionados = mention_df.sort_values(by='Count', ascending=False).head(10)
    formatted_result = [(row['User'], row['Count']) for _, row in top_10_mencionados.iterrows()]
      

    return formatted_result

if __name__ == "__main__":
    file_path = '/Users/edwardguzman/Desktop/challenge_DE/farmers-protest-tweets-2021-2-4.json'
    result = q3_memory(file_path)
    print(result)

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     5     98.4 MiB     98.4 MiB           1   @profile
     6                                         def q3_memory(file_path: str):
     7                                           
     8                                         
     9   1550.7 MiB   1452.3 MiB           1   dataframe = pd.read_json(file_path, lines=True)
    10   1550.7 MiB      0.0 MiB           1   tweets = dataframe['content']
    11   1550.7 MiB      0.0 MiB           1   mention_counts = {}
    12                                         
    13   1551.0 MiB -86270.1 MiB      117408   for tweet in tweets:
    14   1551.0 MiB -86268.7 MiB      117407   mentions = re.findall(r'@(\w+)', tweet)
    15   1551.0 MiB -162634.9 MiB      221473  for mention in mentions:
    16   1551.0 MiB -76364.8 MiB      104066   if mention in mention_counts:
    17   1551.0 MiB -66847.7 MiB       88391   mention_counts[mention] += 1
    18                                         else:
    19   1551.0 MiB  -9518.1 MiB       15675   mention_counts[mention] = 1
    20                                         
    21                                         
    22   1530.1 MiB    -20.9 MiB           1   mention_df = pd.DataFrame(list(mention_counts.items()), columns=['User', 'Count'])
    23   1530.5 MiB      0.5 MiB           1   top_10_mencionados = mention_df.sort_values(by='Count', ascending=False).head(10)
    24   1530.5 MiB      0.0 MiB          13   formatted_result = [(row['User'], row['Count']) for _, row in top_10_mencionados.iterrows()]
    25                                             
    26   1530.5 MiB      0.0 MiB           1   return formatted_result

Ahora realizamos la optimización para revisar si tenemos un mejor rendimiento de memoria. Evitamos la creación de un diccionario mention_counts para almacenar todas las menciones y sus recuentos y mas bien utilizamos collections.Counter para contar las menciones directamente mientras iteramos a través de los tweets. Además, podemos usar Counter.most_common(10) para obtener directamente las 10 menciones más comunes.

In [ ]:
from collections import Counter

def q3_memory_optimized(file_path: str):

    dataframe = pd.read_json(file_path, lines=True)
    tweets = dataframe['content']
    
    mention_counts = Counter()
    for tweet in tweets:
        mentions = re.findall(r'@(\w+)', tweet)
        mention_counts.update(mentions)
    
    top_10_mencionados = mention_counts.most_common(10)
    
    return top_10_mencionados


Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     7     98.3 MiB     98.3 MiB           1   @profile
     8                                         def q3_memory_optimized(file_path: str):
     9                                             
    10                                         
    11   1512.5 MiB   1464.1 MiB           1       dataframe = pd.read_json(file_path, lines=True)
    12   1512.5 MiB      0.0 MiB           1       tweets = dataframe['content']
    13   1512.5 MiB      0.0 MiB           1       mention_counts = Counter()
    14                                         
    15   1512.7 MiB -86270.1 MiB      117408       for tweet in tweets:
    16   1512.7 MiB -86268.7 MiB      117407           mentions = re.findall(r'@(\w+)', tweet)
    17   1512.7 MiB -86269.9 MiB      117407           mention_counts.update(mentions)
    18                                             
    19   1501.5 MiB     -1.2 MiB           1       top_10_mencionados = mention_counts.most_common(10)
    20                                             
    21                                             
    22   1501.5 MiB      0.0 MiB           1       return top_10_mencionados